In [2]:
import geopandas
import numpy as np
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from sklearn import preprocessing, datasets
import matplotlib.pyplot as plt
import csv
import os
from sklearn.metrics import pairwise_distances_argmin
import pandas as pd
from scipy.spatial.distance import cdist,pdist
import itertools
from mpl_toolkits.basemap import Basemap
import math
from scipy import stats
from scipy.sparse import *

In [3]:
os.getcwd()

'/Users/qiancao/Documents'

In [4]:
g = open('file.csv')
csv_f = csv.reader(g,None)
next(csv_f,None)

XX=[]
for row in csv_f:
    row[0],row[1],row[2] = float(row[0]),float(row[1]),float(row[2])
    XX.append(row)

XX = np.array(XX)
X=XX[:,[0,1]]

## I. plotting the sites status quo
#### Finding outstanding sites, like Indonesia has sites plotted in the ocean

In [ ]:
plt.figure(figsize=(12,12))
mm = Basemap(llcrnrlon = min(X[:,0])-10,llcrnrlat = min(X[:,1])-10, urcrnrlon = max(X[:,0])+10,urcrnrlat = max(X[:,1])+10)
mm.drawmapboundary(fill_color = '#A6CAE0',linewidth = 0)
mm.fillcontinents(color = 'grey', alpha = 0.4, lake_color = 'grey')
mm.drawcoastlines(linewidth = 0.5,color = 'white')

plt.scatter(X[:,0],X[:,1],s=80,cmap='viridis')
plt.show()

## II. Clustering w/ Center of Gravity Model

In [15]:
# Assign random clusters as hyperparameter initially, will be optimized by model iteration later

## k: number of clusters
## X: all sites array
## sla: period from despatching to destination
## kmh: kilometer per hour for driving speed
## h: driver driving time per day
## q: sites coverage by such policy (percent)

def model_Kmeans(k,X,sla=2,kmh=60,h=8,q=.9):
    def find_clusters (X, n_clusters, rseed=2):
        #1.Randomly choose clusters
        rng = np.random.RandomState(rseed)
        i = rng.permutation(X.shape[0])[:n_clusters]
        centers = X[i].astype(float)
        
        while True:
            #2a. Assign labels based on closest center: if 5 centers, then labels have 0,1,2,3,4
            labels = pairwise_distances_argmin(X,centers)
            #2b. Find new centers from means of points
            new_centers = np.array([X[labels ==i].mean(0) for i in range(n_clusters)])
            #2c. Check for convergence
            if np.all(centers == new_centers):
                break
            centers = new_centers
        return centers, labels
    centers,labels = find_clusters(X,k)
    # So far the centers are fixed, regardless random centers changes.
    
    # Add each coordination to clusters, which is a dictionary {item(label), X_coordinate}
    clusters = {}
    n=0
    for item in labels:
        if item in clusters:
            clusters[item].append(X[n])
        else:
            clusters[item] = [X[n]]
        n+=1
        
    ### Build distance matrix
    def haversine(lon1,lat1,lon2,lat2):
        """
        Calculate the great circle distance between 2 points
        on the earch (specifid in decimal degree)
        """
        # Convert decimal degree to radians
        lon1,lat1,lon2,lat2 = map(np.radians,[lon1,lat1,lon2,lat2])
        # Haversine formula
        dlon = lon2-lon1
        dlat = lat2-lat1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.math.asin(np.sqrt(a))
        #r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        km = 6367 * c
        return km
    
    a=[]
    b=[]
    
    for i in range (k):
        for j in (clusters[i]):
            a.append(j)
            
    for m in range(k):
        for n in range (len(clusters[m])):
            b.append(centers[m])
            
    matrix1 = pd.DataFrame(a,columns=['CENT_LON','CENT_LAT'])
    matrix2 = pd.DataFrame(b,columns=['CLST_LON','CLST_LAT'])
    matrix  = pd.concat([matrix1,matrix2],axis=1,join_axes=[matrix1.index])
    
    for i in matrix.index:
        matrix.loc[i,'CENT_SITE_DIST'] = 1.19*haversine(matrix.loc[i,'CENT_LON'],matrix.loc[i,'CENT_LAT'],matrix.loc[i,'CLST_LON'],matrix.loc[i,'CLST_LAT'])
    
    # Drop duplicated site for avoiding dulicate CBM mapping
    matrix.drop_duplicates(subset=['CENT_LON','CENT_LAT'],inplace=True)
    matrix.reset_index(drop = True,inplace = True)
    
    # Processing the dataset for adding volume
    df_v = pd.DataFrame(XX,columns=['ORG_LON','ORG_LAT','CBM'])
    df_v.CBM = df_v.CBM.apply(lambda x: float(x))
    df_v = df_v.groupby(['ORG_LON','ORG_LAT'])['CBM'].sum()
    df_gpv = pd.DataFrame(df_v).reset_index()
    
    matrix_V = pd.merge(matrix,df_gpv,left_on=['CENT_LON','CENT_LAT'],right_on=['ORG_LON','ORG_LAT'])
    matrix_V.drop(['CENT_LON','CENT_LAT'],axis = 1, inplace = True)
    
    # The center-of-gravity method
    # Cx = Sum(DixVi)/Sum(Vi) AND Cy = Sum(DiyVi)/Sum(Vi)
    # Parameters: 
    # Cx--重心的X坐标 
    # Cy--重心的Y坐标
    # Dix--第i个地点的X坐标
    # Diy--第i个地点的Y坐标
    # Vi--运到第i个地点，或从第i地点运出的货物量

    matrix_V_gb = matrix_V.groupby(['CLST_LON','CLST_LAT'])
    
    # Calculating the Center of Gravity
    def COG(gp):
        LON = sum(np.multiply(gp['ORG_LON'],gp['CBM']))/sum(gp['CBM'])
        LAT = sum(np.multiply(gp['ORG_LAT'],gp['CBM']))/sum(gp['CBM'])
        return LON,LAT
    
    # Adding the new-COG to the clusted diagram list
    Re_center = matrix_V_gb.apply(lambda grp: COG(grp)[0])
    matrix_COG = pd.DataFrame(Re_center).reset_index()
    matrix_COG.columns = ['CLST_LON','CLST_LAT','COG_LON']
    centerList=matrix_V_gb.apply(lambda grp: COG(grp)[1]).values
    matrix_COG['COG_LAT'] = centerList
    
    matrix_New = pd.merge(matrix_V,matrix_COG, on=['CLST_LON','CLST_LAT'],how = 'left')
    matrix_New = matrix_New[['ORG_LON','ORG_LAT','COG_LON','COG_LAT','CBM']]
    
    for i in matrix_New.index:
        matrix_New.loc[i,'DISTANCE']= 1.19*haversine(matrix_New.loc[i,'ORG_LON'],matrix_New.loc[i,'ORG_LAT'],matrix_New.loc[i,'COG_LON'],matrix_New.loc[i,'COG_LAT'])
        
        
    # Sort distance ascending by clusters
    m_min = matrix_New.sort_values(by=['COG_LON','COG_LAT','DISTANCE']).groupby(['COG_LON','COG_LAT']).head(1)
    
    # Apply closest site to the Center Location
    def replace_COG(df):
        test=m_min[(m_min.COG_LON == df['COG_LON']) & (m_min.COG_LAT == df['COG_LAT'])]
        df['COG_LON']=test['ORG_LON']
        df['COG_LAT']=test['ORG_LAT']
        return df
    
    matrix_New.apply(replace_COG,axis=1)
    
    # Stacking LON and LAT to one array
    Centroids_New = np.column_stack((matrix_New.COG_LON.unique(),matrix_New.COG_LAT.unique()))
    SLA = matrix_New.groupby(['COG_LON','COG_LAT'])['DISTANCE'].quantile(q)/kmh/h
    
    # Calculating the SLA
    matrix_New['SLA'] = matrix_New['DISTANCE'].apply(lambda x: math.ceil(float(x/kmh)/h))
    matrix_New.sort_values(['COG_LON','COG_LAT','SLA'],ascending = [False,False,True],inplace = True)
    
    percentile = stats.percentileofscore(matrix_New['SLA'],sla+1,kind='rank')
    return SLA, percentile,centers,matrix_New,Centroids_New,labels
        

## III. Run Model: finding conditions complying with required SLA:
### Step1. Initialization: 
Hyperparameter: SLA limit, Tranportation speed, Working Hours, Coverage by Percent

In [8]:
SLA = 0.5
kmh = 60
h = 8
q = 0.9

### Step2. Find out proper K of clusters satisfying SLA required

In [16]:
for k in range (1,10):
    p = model_Kmeans(k,X,SLA,kmh,h)[1]
    s = model_Kmeans(k,X,SLA,kmh,h)[0]
    print (k, 'centers',SLA, 'day(s) covers', round(p,1), '% sites')
    if (max(s.values)>SLA) | (p<90):
        print (round(max(s.values),2),'day(s) covers', q*100, '% sites')
        continue
    else:
        print (round(max(s.values),2),'day(s) covers',round(p,1), '% sites')
        break

1 centers 0.5 day(s) covers 5.9 % sites
4.36 day(s) covers 90.0 % sites
2 centers 0.5 day(s) covers 23.5 % sites
4.44 day(s) covers 90.0 % sites
3 centers 0.5 day(s) covers 26.6 % sites
2.79 day(s) covers 90.0 % sites
4 centers 0.5 day(s) covers 37.7 % sites
2.79 day(s) covers 90.0 % sites
5 centers 0.5 day(s) covers 49.9 % sites
2.59 day(s) covers 90.0 % sites
6 centers 0.5 day(s) covers 54.8 % sites
2.57 day(s) covers 90.0 % sites
7 centers 0.5 day(s) covers 58.9 % sites
2.22 day(s) covers 90.0 % sites
8 centers 0.5 day(s) covers 63.0 % sites
1.88 day(s) covers 90.0 % sites
9 centers 0.5 day(s) covers 72.1 % sites
1.88 day(s) covers 90.0 % sites


In [17]:
# Generate SLA, Distance, Centers, Dataframe w/COG, Nearest Site as Center, Classes
s,p,c,m,t,l = model_Kmeans(k,X,SLA,kmh,h)

In [18]:
m.sort_values(['COG_LON','COG_LAT','SLA'],ascending=[False,False,True])

,ORG_LON,ORG_LAT,COG_LON,COG_LAT,CBM,DISTANCE,SLA
133,-35.0300,-8.2900,-37.0700,-8.4200,6825.0,321.170625,1
134,-37.0700,-8.4200,-37.0700,-8.4200,6027.0,62.492410,1
135,-37.3400,-5.1900,-37.0700,-8.4200,4359.0,395.855624,1
136,-37.4500,-11.2696,-37.0700,-8.4200,1452.0,408.561023,1
138,-35.0200,-8.1100,-37.0700,-8.4200,8267.0,322.357450,1
139,-38.8500,-6.4000,-37.0700,-8.4200,5781.0,296.035328,1
140,-34.8500,-8.0000,-37.0700,-8.4200,257.0,345.347578,1
141,-34.9156,-8.0756,-37.0700,-8.4200,1035.0,336.189210,1
142,-36.9101,-5.5796,-37.0700,-8.4200,3083.0,351.984787,1
143,-38.2657,-9.3307,-37.0700,-8.4200,7329.0,183.443251,1


##  IV. Make-up the map displaying sites

In [1]:
plt.figure(figsize=(12,12))
mp = Basemap(llcrnrlon = min(m.ORG_LON)-10,llcrnrlat = min(m.ORG_LAT)-10, urcrnrlon = max(m.ORG_LON)+10,urcrnrlat = max(m.ORG_LAT)+10)
mp.drawmapboundary(fill_color = '#A6CAE0',linewidth = 0)
mp.fillcontinents(color = 'grey', alpha = 0.4, lake_color = 'grey')
mp.drawcoastlines(linewidth = 0.5,color = 'white')

plt.scatter(X[:,0],X[:,1],c=1,s=80,cmap='viridis')
plt.scatter(c[:,0],X[:,1],c='white',s=200,alpha=1)

plt.show()

NameError: name 'plt' is not defined